In [ ]:
import pandas as pd

perfs = ["ultrabullet", "bullet", "blitz", "rapid", "classical"]

dtypes = {"username": "string"}

for perf in perfs:
    dtypes[f"{perf}_approximate_time"] = "Int64"
    dtypes[f"{perf}_real_time"] = "Int64"


df = pd.read_csv("time-spent-2017-04.csv",dtype=dtypes)
df.dtypes

In [ ]:
df_time = df.copy()

def convert(c):
    df_time[c] = df_time[c].apply(lambda x: pd.to_timedelta(x,unit="S"))

for perf in perfs:
    convert(f"{perf}_approximate_time")
    convert(f"{perf}_real_time")
df_time

In [ ]:
df_time.describe()

In [ ]:
df1 = df[["username", "bullet_games", "bullet_approximate_time", "bullet_real_time"]].copy()
df1.sort_values("bullet_real_time", ascending=False, inplace=True)
df1

In [ ]:
df1.hist(bins=1000,log=True)

In [ ]:
# Only show players with more than x games
x = 100
more_x_bullet_games = df1.query(f"bullet_games>{x}").copy()
print(f"{more_x_bullet_games.shape[0]} players with more than {x} games")
more_x_bullet_games.hist(bins=1000)

In [ ]:
colors = [ "#1f77b4","#ff7f0e","#2ca02c","#d62728","#9467bd","#8c564b","#e377c2","#7f7f7f","#bcbd22","#17becf"]
for (perf, color) in zip(perfs, colors):
    df.hist(bins=1000,log=True,color=color,column=[f"{perf}_games", f"{perf}_approximate_time", f"{perf}_real_time"])

In [ ]:
# Compute the relative error of the real time spent, compared to the average computed one
df_avg_error = df.copy()
for perf in perfs:
    df_avg_error[f"{perf}_relative_error"] = (df_avg_error[f"{perf}_approximate_time"] - df_avg_error[f"{perf}_real_time"]).abs() / df_avg_error[f"{perf}_real_time"]

# Restricting to blitz to see all columns
df_avg_error[["username", "blitz_games", "blitz_approximate_time", "blitz_real_time", "blitz_relative_error"]]

In [ ]:
df_avg_error.dtypes

In [ ]:
df_avg_error[[f"{p}_relative_error" for p in perfs]].describe()

In [ ]:
# Investing why the max error is so high

max_avg_error_first = df_avg_error.sort_values("blitz_relative_error", ascending=False)
max_avg_error_first.query("blitz_games>2")[["username", "blitz_games", "blitz_approximate_time", "blitz_real_time", "blitz_relative_error"]][:20]

In [ ]:
# Only show players with more than x games
x = 100
for (perf, color) in zip(perfs, colors):
    more_x_games = df_avg_error.query(f"{perf}_games>{x}").copy()
    print(f"{more_x_games.shape[0]} players with more than {x} {perf} games")
    more_x_games.hist(bins=100,column=f"{perf}_relative_error",color=color)